<a href="https://colab.research.google.com/github/Orley74/Przetwarzanie_jezyka_naturalnego/blob/main/PJNLAB5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run this cell to import packages.
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# pulling resources necessary to run code
# they will be stored in '/content/nlp-resources-edu/05_Embedding_lab/' folder

!git clone  https://github.com/mmazurek-wat/nlp-resources-edu.git
resource_path = '/content/nlp-resources-edu/05_Embedding_lab/'

sys.path.append(resource_path)

Cloning into 'nlp-resources-edu'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 3), reused 18 (delta 0), pack-reused 0
Receiving objects: 100% (25/25), 3.75 MiB | 3.50 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
from os import path
capitals_file = 'capitals.txt'
capitals_file_path = path.join(resource_path, capitals_file)
data = pd.read_csv(capitals_file_path, delimiter=' ')
data.columns = ['city1', 'country1', 'city2', 'country2']

# print first five elements in the DataFrame
data.head(5)

,city1,country1,city2,country2
0,Athens,Greece,Bangkok,Thailand
1,Athens,Greece,Beijing,China
2,Athens,Greece,Berlin,Germany
3,Athens,Greece,Bern,Switzerland
4,Athens,Greece,Cairo,Egypt


In [ ]:
import gensim.downloader as api

embeddings = api.load('word2vec-google-news-300')

vec_king = embeddings['king']

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
import nltk
from gensim.models import KeyedVectors


#embeddings = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary = True)

#embedding as dictionary


word_embeddings = {}
for word in embeddings.key_to_index:
  word_embeddings[word] = embeddings[word]

print(len(word_embeddings))

FileNotFoundError: ignored

In [ ]:
embedding_subset_file = 'word_embeddings_subset.p'
embedding_subset_file_path = path.join(resource_path, embedding_subset_file)

word_embeddings = pickle.load(open(embedding_subset_file_path, "rb"))
len(word_embeddings)  # there should be 243 words that will be used in this assignment

243

In [ ]:
print(* word_embeddings.keys())

country city China Iraq oil town Canada London England Australia Japan Pakistan Iran gas happy Russia Afghanistan France Germany Georgia Baghdad village Spain Italy Beijing Jordan Paris Ireland Turkey Egypt Lebanon Taiwan Tokyo Nigeria Vietnam Moscow Greece Indonesia sad Syria Thailand Libya Zimbabwe Cuba Ottawa Tehran Sudan Kenya Philippines Sweden Poland Ukraine Rome Venezuela Switzerland Berlin Bangladesh Portugal Ghana Athens king Madrid Somalia Dublin Qatar Chile Islamabad Bahrain Nepal Norway Serbia Kabul continent Brussels Belgium Uganda petroleum Cairo Denmark Austria Jamaica Georgetown Bangkok Finland Peru Romania Bulgaria Hungary Vienna Kingston Manila Cyprus Azerbaijan Copenhagen Fiji Tunisia Kazakhstan queen Beirut Jakarta Croatia Belarus Algeria Malta Morocco Rwanda Bahamas Damascus Ecuador Angola Canberra Liberia Honduras Tripoli Slovakia Doha Armenia Taipei Oman Nairobi Santiago Guinea Uruguay Stockholm Slovenia Zambia Havana Uzbekistan Belgrade Mogadishu Khartoum Botswa

In [ ]:
print("dimension: {}".format(word_embeddings['Spain'].shape[0]))

dimension: 300


In [ ]:
# use numpy.linalg.norm , numpy.dot


def cosine_similarity(A, B):
    '''
    Input:
        A: a numpy array which corresponds to a word vector
        B: A numpy array which corresponds to a word vector
    Output:
        cos: numerical number representing the cosine similarity between A and B.
    '''

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    dot = np.dot(A, B)
    norma = np.linalg.norm(A)
    normb = np.linalg.norm(B)
    cos = dot / (norma * normb)

    ### END CODE HERE ###
    return cos

In [ ]:
# feel free to try different words
king = word_embeddings['king']
queen = word_embeddings['queen']

cosine_similarity(king, queen)

0.6510956

In [ ]:
# use numpy.linalg.norm

def euclidean(A, B):
    """
    Input:
        A: a numpy array which corresponds to a word vector
        B: A numpy array which corresponds to a word vector
    Output:
        d: numerical number representing the Euclidean distance between A and B.
    """

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    # euclidean distance

    d = np.linalg.norm(A - B)

    ### END CODE HERE ###

    return d

In [ ]:
# Test your function
euclidean(king, queen)

2.4796925

In [ ]:
def get_country(city1, country1, city2, embeddings):
    """
    Input:
        city1: a string (the capital city of country1)
        country1: a string (the country of capital1)
        city2: a string (the capital city of country2)
        embeddings: a dictionary where the keys are words and values are their embeddings
    Output:
        countries: a dictionary with the most likely country and its similarity score
    """
    ### START CODE HERE ###

    # store the city1, country 1, and city 2 in a set called group
    group = set((city1, country1, city2))

    # get embeddings of city 1
    city1_emb = embeddings.get(city1, None)

    # get embedding of country 1
    country1_emb = embeddings.get(country1, None)

    # get embedding of city 2
    city2_emb = embeddings.get(city2, None)

    # get embedding of country 2 (it's a combination of the embeddings of country 1, city 1, and city 2)
    # Remember: King - Man + Woman = Queen
    vec = country1_emb - city1_emb + city2_emb

    # Initialize the similarity to -1 (it will be replaced by similarities that are closer to +1)
    similarity = -1

    # initialize country to an empty string (you will return it as a tuple)
    country = ''

    # loop through all words in the embeddings dictionary
    for word in embeddings.keys():

        # first check that the word is not already in the 'group' (it is not a part of input dataset)
        if word not in group:

            # get the word embedding
            word_emb = embeddings[word]

            # calculate cosine similarity between the embedding of country 2 and the word in the embeddings dictionary
            cur_similarity = cosine_similarity(vec, word_emb)

            # if the cosine similarity is more similar than the previously best similarity...
            if cur_similarity > similarity:

                # update the similarity to the new, better similarity
                similarity = cur_similarity

                # store the country as a tuple, which contains the word and the similarity
                country = (word, similarity)

    ### END CODE HERE ###

    return country

In [ ]:
# Testing your function, note to make it more robust you can return the 5 most similar words.
get_country('Athens', 'Greece', 'Cairo', word_embeddings)

('Egypt', 0.7626821)

In [ ]:
def get_accuracy(word_embeddings, data):
    '''
    Input:
        word_embeddings: a dictionary where the key is a word and the value is its embedding
        data: a pandas dataframe containing all the country and capital city pairs

    Output:
        accuracy: the accuracy of the model
    '''

    ### START CODE HERE ###

    # initialize num correct to zero
    num_correct = 0

    # loop through the rows of the dataframe
    for i, row in data.iterrows():

        # get city1
        city1 = row['city1']

        # get country1
        country1 = row['country1']

        # get city2
        city2 = row['city2']

        # get country2
        country2 = row['country2']

        # use get_country to find the predicted country2
        predicted_country2, _ = get_country(city1, country1, city2, word_embeddings)

        # if the predicted country2 is the same as the actual country2...
        if predicted_country2 == country2:
            # increment the number of correct by 1
            num_correct += 1

    # get the number of rows in the data dataframe (length of dataframe)
    m = len(data)

    # calculate the accuracy by dividing the number correct by m
    accuracy = num_correct / m

    ### END CODE HERE ###
    return accuracy



In [ ]:
accuracy = get_accuracy(word_embeddings, data)
print(f"Accuracy is {accuracy:.2f}")

Accuracy is 0.92


In [ ]:
def find_closest_word(v, excluded_word, embeddings):
    closest_word = None
    min_distance = float('inf')

    for word in embeddings.index_to_key:
        if word == excluded_word:
            continue

        word_emb = embeddings[word]
        distance = np.linalg.norm(v - word_emb)

        if distance < min_distance:
            closest_word = word
            min_distance = distance
    return closest_word

In [ ]:
rel_src1 = 'fast'
rel_trg1 = 'slow'
rel_src2 = 'clever'
test_vector = embeddings[rel_src1] - embeddings[rel_trg1] + embeddings[rel_src2]
def get_analogy(rel_src1, rel_trg1, rel_src2, embeddings):
    # Calculate the vector representation of the analogy
    analogy_vector = embeddings[rel_trg1] - embeddings[rel_src1] + embeddings[rel_src2]

    # Find the closest word to the analogy vector, excluding the source word
    rel_trg2 = find_closest_word(analogy_vector, rel_src2, embeddings)

    return rel_trg2

In [ ]:
result = get_analogy(rel_src1, rel_trg1, rel_src2, embeddings)

print(f"{rel_src1} is to {rel_trg1} as {rel_src2} is to {result}")

fast is to slow as clever is to clumsy


In [ ]:
rel=('fast', 'slow', 'clever')  #fast to slow is like clever to